In [11]:
# Import relevant libraries.
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [12]:
# Change working directory to 2HRM folder.
# cwd = os.getcwd()
# print(cwd)
os.chdir(r'C:\Users\l619072\Downloads\2HQI')
cwd = os.getcwd()
print(cwd)

C:\Users\l619072\Downloads\2HQI


In [13]:
# Load inventory report. Make sure to sort by batch AND item code in excel first!
lot_history = pd.read_csv(r'C:\Users\l619072\Downloads\2HQI\2HQI Inventory Report - Copy.csv')
# lot_history = lot_history.fillna(0)
lot_history.head()

,Item Code,P I D,OH_QTY,ON_WGT_N
0,100013202,21116,8,637.82
1,100013202,21116,20,1594.54
2,100013202,21116,24,1913.45
3,100013202,21116,25,1993.18
4,100013202,21116,54,2152.63


In [14]:
current_date = datetime.now().strftime('%m/%d/%Y')
current_time = datetime.now().time().strftime("%H:%M %p")
# print(current_date)
# print(current_time)

In [15]:
# Test run to see what each row looks like in python.
for index,row in lot_history.iterrows():
    if index > 0:
        break
    print(row)
    print(row[0])
    print(row[1])
    print(row[3])

Item Code    100013202
P I D            21116
OH_QTY               8
ON_WGT_N        637.82
Name: 0, dtype: object
100013202
21116
637.82


In [16]:
print(lot_history['Item Code'][0])
print(lot_history['P I D'][0])
print(lot_history['OH_QTY'][0])
print(lot_history['ON_WGT_N'][0])

100013202
21116
8
637.82


In [17]:
recon_list = [] # It's easier to build up a list and then create the dataframe
counter = 0 # Index for recon_list because we only want to increment recon_list's index if we reach a new batch number.

for index,row in lot_history.iterrows():
    if len(recon_list) == 0: # If test is empty, then initialize the process
        recon_list.append([current_date, current_time, '2HQI', lot_history['Item Code'][index], '(blank)', lot_history['P I D'][index], 'F', '(blank)', lot_history['OH_QTY'][index], 'CV', lot_history['ON_WGT_N'][index], 'A100'])
        continue
    if row[1] == recon_list[counter][5] and row[0] == recon_list[counter][3]: # If row's batch number matches recon_list's current batch number.
        recon_list[counter][8] += row[2] # sum CV
        recon_list[counter][10] += row[3] # sum LB
    else: # If we hit a new batch, then increment recon_list's index and create new row for new batch.
        counter += 1
        recon_list.append([current_date, current_time, '2HQI', lot_history['Item Code'][index], '(blank)', lot_history['P I D'][index], 'F', '(blank)', lot_history['OH_QTY'][index], 'CV', lot_history['ON_WGT_N'][index], 'A100'])
        
df = pd.DataFrame(recon_list, columns = ['Snapshot Date', 'Snapshot Time', 'Plant ID', 'Product Code', 'Product Description', 'Batch Number', 'Stock Type', 'Pallet ID', 'Available Qty.', 'UOM', 'Weight(Mandatory)', 'Stor.Loc'])
df

,Snapshot Date,Snapshot Time,Plant ID,Product Code,Product Description,Batch Number,Stock Type,Pallet ID,Available Qty.,UOM,Weight(Mandatory),Stor.Loc
0,03/25/2022,09:20 AM,2HQI,100013202,(blank),21116,F,(blank),159,CV,10523.98,A100
1,03/25/2022,09:20 AM,2HQI,TK15,(blank),100701,F,(blank),21,CV,42.00,A100
2,03/25/2022,09:20 AM,2HQI,XC34,(blank),100701,F,(blank),50,CV,100.00,A100
3,03/25/2022,09:20 AM,2HQI,PC30,(blank),111104,F,(blank),22,CV,44.00,A100
4,03/25/2022,09:20 AM,2HQI,TK15,(blank),120412,F,(blank),54,CV,108.00,A100
...,...,...,...,...,...,...,...,...,...,...,...,...
2088,03/25/2022,09:20 AM,2HQI,103573436,(blank),36421DPF30,F,(blank),487,CV,9680.00,A100
2089,03/25/2022,09:20 AM,2HQI,103585646,(blank),36520DOY30,F,(blank),15,CV,87.32,A100
2090,03/25/2022,09:20 AM,2HQI,103585656,(blank),36520DOY30,F,(blank),7,CV,48.10,A100
2091,03/25/2022,09:20 AM,2HQI,103576867,(blank),36520DPF30,F,(blank),77,CV,2310.00,A100


In [18]:
# Export dataframe as excel in current working directory
file_name = '2HQI ZEWM10 upload.xlsx'
df.to_excel(file_name, index=False)
print('DataFrame is successfully exported as an excel file!')

DataFrame is successfully exported as an excel file!


In [42]:
# Compare case count from python created excel and manually created excel
df1 = pd.read_excel(r'C:\Users\l619072\Downloads\2HQI\Book1.xlsx')
df2 = pd.read_excel(r'C:\Users\l619072\Downloads\2HQI\2HQI ZEWM10 upload.xlsx')

# add the cases from df2 to df1
df1['CV_Count'] = df2['Available Qty.']
 
# create new column in df1 to check if cases match
df1['CV_Matching'] = np.where(df1['CV_Count'] == df2['Available Qty.'], 'True', 'False') 
df1

,Snapshot Date,Snapshot Time,Plant ID,Product Code,Product Description,Batch Number,Stock Type,Pallet ID,Available Qty.,UOM,Weight (Mandatory),Stor.Loc,CV_Count,CV_Matching
0,2022-03-25,13:30:00 PM,2HQI,100013202,(blank),21116,F,(blank),159,CV,10523.98,A100,159,True
1,2022-03-25,13:30:00 PM,2HQI,TK15,(blank),100701,F,(blank),21,CV,42.00,A100,21,True
2,2022-03-25,13:30:00 PM,2HQI,XC34,(blank),100701,F,(blank),50,CV,100.00,A100,50,True
3,2022-03-25,13:30:00 PM,2HQI,PC30,(blank),111104,F,(blank),22,CV,44.00,A100,22,True
4,2022-03-25,13:30:00 PM,2HQI,TK15,(blank),120412,F,(blank),54,CV,108.00,A100,54,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088,2022-03-25,13:30:00 PM,2HQI,103573436,(blank),36421DPF30,F,(blank),487,CV,9680.00,A100,487,True
2089,2022-03-25,13:30:00 PM,2HQI,103585646,(blank),36520DOY30,F,(blank),15,CV,87.32,A100,15,True
2090,2022-03-25,13:30:00 PM,2HQI,103585656,(blank),36520DOY30,F,(blank),7,CV,48.10,A100,7,True
2091,2022-03-25,13:30:00 PM,2HQI,103576867,(blank),36520DPF30,F,(blank),77,CV,2310.00,A100,77,True


In [43]:
# Export dataframe as excel in current working directory
file_name = 'Check_matching_CV.xlsx'
df1.to_excel(file_name, index=False)
print('DataFrame is successfully exported as an excel file!')

DataFrame is successfully exported as an excel file!
